In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

In [2]:
df = pd.read_csv("/Users/gblasd/Documents/DataScience/02_SupervisedLearning/data/OnlineNewsPopularity/OnlineNewsPopularity.csv").reset_index(drop = True)

In [3]:
df.columns = [x.strip() for x in df.columns]

In [4]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [5]:
df['shares'].describe(percentiles=[.01,.25,.5,.75,.99])

count     39644.000000
mean       3395.380184
std       11626.950749
min           1.000000
1%          381.000000
25%         946.000000
50%        1400.000000
75%        2800.000000
99%       31657.000000
max      843300.000000
Name: shares, dtype: float64

In [6]:
df["success"] = (df["shares"] > 5000)*1

In [7]:
df['success'].value_counts(1)

success
0    0.872061
1    0.127939
Name: proportion, dtype: float64

In [8]:
ls_cont = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']
target = "shares"
target_disc = "success"

In [9]:
X = df[ls_cont]
yc = df[target_disc]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, yc)

In [11]:
X_train.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
20268,10.0,1672.0,0.262991,1.0,0.312551,3.0,3.0,32.0,0.0,4.956340,...,0.317460,0.050000,1.0,-0.125962,-0.4,-0.100000,0.5,0.5,0.0,0.5
1984,9.0,682.0,0.439060,1.0,0.641509,7.0,2.0,9.0,0.0,4.599707,...,0.408788,0.136364,1.0,-0.215903,-0.5,-0.050000,0.0,0.0,0.5,0.0
15547,6.0,835.0,0.477887,1.0,0.683128,21.0,4.0,11.0,0.0,4.506587,...,0.399394,0.100000,0.8,-0.312344,-1.0,-0.071429,0.5,0.5,0.0,0.5
24445,11.0,1694.0,0.470588,1.0,0.613491,10.0,7.0,51.0,1.0,4.649351,...,0.425643,0.062500,1.0,-0.301790,-0.8,-0.050000,0.5,0.3,0.0,0.3
11539,13.0,729.0,0.490085,1.0,0.613169,10.0,0.0,1.0,0.0,5.155007,...,0.340873,0.100000,0.6,-0.307143,-0.4,-0.071429,0.0,0.0,0.5,0.0


## Linear Discriminant Analysis (LDA)

_Linear Discriminant Analysis_ (LDA) is actually a classification algorithm, but durin training it learns the most discriminative axes between the classes, and these benefit is that the projection willl keep classes as far apart as possible, so LDA is a good technique to reduce dimensionality before running another classification algorithm such as an SVM classifier.

In [12]:
lda = LinearDiscriminantAnalysis()

In [13]:
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [14]:
ls_res = cross_val_score(estimator = lda, X=X_train, y=y_train, cv=4, n_jobs=-1, scoring="roc_auc")

In [15]:
ls_res

array([0.68395743, 0.68024769, 0.68866104, 0.69101197])

In [16]:
np.mean(ls_res), np.std(ls_res)

(np.float64(0.6859695333377971), np.float64(0.004167084933143411))

In [17]:
LinearDiscriminantAnalysis?

Init signature:
LinearDiscriminantAnalysis(
    solver='svd',
    shrinkage=None,
    priors=None,
    n_components=None,
    store_covariance=False,
    tol=0.0001,
    covariance_estimator=None,
)
Docstring:     
Linear Discriminant Analysis.

A classifier with a linear decision boundary, generated by fitting class
conditional densities to the data and using Bayes' rule.

The model fits a Gaussian density to each class, assuming that all classes
share the same covariance matrix.

The fitted model can also be used to reduce the dimensionality of the input
by projecting it to the most discriminative directions, using the
`transform` method.

.. versionadded:: 0.17

For a comparison between
:class:`~sklearn.discriminant_analysis.LinearDiscriminantAnalysis`
and :class:`~sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis`, see
:ref:`sphx_glr_auto_examples_classification_plot_lda_qda.py`.

Read more in the :ref:`User Guide <lda_qda>`.

Parameters
----------
solver : {'svd', 'lsqr'

In [18]:
lda.get_params()

{'covariance_estimator': None,
 'n_components': None,
 'priors': None,
 'shrinkage': None,
 'solver': 'svd',
 'store_covariance': False,
 'tol': 0.0001}

In [19]:
param_grid = {"solver": ["svd", "lsqr", "eigen"],
              "shrinkage": [None, "auto"] + [x/100 for x in range(100)]+[x for x in range(100)]}

In [20]:
np.prod(list(map(len, param_grid.values())))

np.int64(606)

### Sin escalar

In [21]:
rs = GridSearchCV(cv=4, error_score=-1000, estimator=lda, n_jobs=-1, scoring="roc_auc", param_grid=param_grid, verbose=True)

In [22]:
%%time
rs.fit(X=X_train, y=y_train)

Fitting 4 folds for each of 606 candidates, totalling 2424 fits
CPU times: user 647 ms, sys: 365 ms, total: 1.01 s
Wall time: 6.18 s


/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
1600 fits failed out of a total of 2424.
The score on these train-test partitions for these parameters will be set to -1000.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gblasd/Documen

GridSearchCV(cv=4, error_score=-1000, estimator=LinearDiscriminantAnalysis(),
             n_jobs=-1,
             param_grid={'shrinkage': [None, 'auto', 0.0, 0.01, 0.02, 0.03,
                                       0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                                       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17,
                                       0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
                                       0.25, 0.26, 0.27, ...],
                         'solver': ['svd', 'lsqr', 'eigen']},
             scoring='roc_auc', verbose=True)

In [23]:
rs.best_score_

np.float64(0.6859695333377971)

### Con Escalamiento

In [24]:
sc = MinMaxScaler()
Xs = sc.fit_transform(X_train)

In [25]:
%%time
rs.fit(X=Xs, y=y_train)

Fitting 4 folds for each of 606 candidates, totalling 2424 fits
CPU times: user 581 ms, sys: 297 ms, total: 878 ms
Wall time: 4.7 s


/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
1594 fits failed out of a total of 2424.
The score on these train-test partitions for these parameters will be set to -1000.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gblasd/Document

GridSearchCV(cv=4, error_score=-1000, estimator=LinearDiscriminantAnalysis(),
             n_jobs=-1,
             param_grid={'shrinkage': [None, 'auto', 0.0, 0.01, 0.02, 0.03,
                                       0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                                       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17,
                                       0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
                                       0.25, 0.26, 0.27, ...],
                         'solver': ['svd', 'lsqr', 'eigen']},
             scoring='roc_auc', verbose=True)

In [26]:
rs.best_score_

np.float64(0.6859695333377971)

In [27]:
rs.best_estimator_

LinearDiscriminantAnalysis()

In [28]:
rs.best_estimator_.coef_[0]

array([ 7.28747229e-02,  9.76853065e-01,  8.85013163e-01, -9.51368016e-01,
       -2.11323007e-01,  4.15371732e+00, -3.06596723e+00,  9.87668818e-01,
        1.12184512e+00, -2.15600404e+00,  2.24417726e-01,  2.33304336e-01,
        1.40353822e+01, -1.43197255e+01, -3.32512894e-01, -9.52150687e-02,
       -9.77068514e-01, -5.33221401e-01, -3.22398954e+01,  3.75108761e+01,
       -4.82809607e-01, -1.29765026e+00,  5.31262270e+00,  1.99209092e-01,
       -2.50963954e-01, -2.06075460e-01, -3.28842928e-03,  1.76106937e-01,
        8.81398404e-01, -9.60540759e-02, -1.14748594e-01, -6.43198917e-01,
        1.09112871e+00,  1.55951419e+00, -1.57546558e-01, -6.28686643e-01,
        8.63056935e-03, -2.81556733e-01, -2.33821672e-02, -4.86456717e-02,
        2.04801058e-01,  4.21859678e-01,  2.43583126e-01,  1.42821488e-01])

In [29]:
pd.DataFrame(zip(ls_cont, rs.best_estimator_.coef_[0])).sort_values(by = 1)

,0,1
18,kw_max_avg,-32.239895
13,kw_avg_min,-14.319726
6,num_self_hrefs,-3.065967
9,average_token_length,-2.156004
21,self_reference_max_shares,-1.297650
16,kw_avg_max,-0.977069
3,n_non_stop_words,-0.951368
31,global_rate_negative_words,-0.643199
35,min_positive_polarity,-0.628687
17,kw_min_avg,-0.533221


In [30]:
rs.best_estimator_.means_

array([[0.37019201, 0.06405998, 0.53247123, 0.97285626, 0.67573564,
        0.03463904, 0.02841444, 0.03347392, 0.01321512, 0.56764634,
        0.68835681, 0.08532711, 0.00377933, 0.00721106, 0.0161283 ,
        0.89067105, 0.30431854, 0.30085773, 0.01829833, 0.07006614,
        0.00526912, 0.01132318, 0.00844478, 0.18570822, 0.13624006,
        0.23102953, 0.21285638, 0.23942682, 0.44154148, 0.45719266,
        0.25443826, 0.11844743, 0.6841032 , 0.28859883, 0.35303451,
        0.09529543, 0.75478243, 0.7425286 , 0.48160882, 0.89316356,
        0.27682954, 0.53447081, 0.68224985, 0.1517614 ],
       [0.36771135, 0.06471896, 0.52414703, 0.95391098, 0.65767706,
        0.04276749, 0.02800759, 0.04599437, 0.0189334 , 0.55415087,
        0.71301905, 0.08227994, 0.00478107, 0.00839737, 0.0184575 ,
        0.9013351 , 0.33166456, 0.3716094 , 0.02359318, 0.0854807 ,
        0.00980099, 0.0190596 , 0.01509002, 0.17945478, 0.12791002,
        0.13520613, 0.33314094, 0.22968456, 0.45712115, 0.4

In [31]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,success
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0


### SelectKBest

In [32]:
len(ls_cont)

44

In [33]:
kb = SelectKBest(k = 2, score_func=f_classif)
Xb = kb.fit_transform(Xs, y_train)

In [34]:
best = [v for v,s in zip(ls_cont,kb.get_support()) if s]
best

['kw_avg_avg', 'LDA_03']

In [35]:
#ls_cont

In [36]:
#kb.get_support()

In [37]:
%%time
rs.fit(X=Xb, y=y_train)

Fitting 4 folds for each of 606 candidates, totalling 2424 fits
CPU times: user 315 ms, sys: 160 ms, total: 475 ms
Wall time: 1.24 s


/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
1588 fits failed out of a total of 2424.
The score on these train-test partitions for these parameters will be set to -1000.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
412 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gblasd/Documents/DataScience/.venv/lib/python3.13/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/gblasd/Docume

GridSearchCV(cv=4, error_score=-1000, estimator=LinearDiscriminantAnalysis(),
             n_jobs=-1,
             param_grid={'shrinkage': [None, 'auto', 0.0, 0.01, 0.02, 0.03,
                                       0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1,
                                       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17,
                                       0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
                                       0.25, 0.26, 0.27, ...],
                         'solver': ['svd', 'lsqr', 'eigen']},
             scoring='roc_auc', verbose=True)

In [38]:
rs.best_score_

np.float64(0.6582172150841321)

In [39]:
pd.DataFrame(zip(best, rs.best_estimator_.coef_[0])).sort_values(by = 1)

,0,1
1,LDA_03,0.573622
0,kw_avg_avg,14.354487


In [40]:
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,success
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0
